In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [0]:
data = pd.read_csv('/content/drive/My Drive/ML 5th sem proj/GALEX_data-extended-feats.csv')

In [0]:
x = data.iloc[:,1:25]
y = data.iloc[:,0]

x = (x.to_numpy())                     #features
y = (y.to_numpy())                     #classes

y = y.reshape(8406,1)

In [0]:
# Standardize the data for
# 1)Faster Convergence
# 2)Since scale of each attribute is not same
# 3)Larger input values dominate the network otherwise

#  Data point with non-zero mean value has a bigger eigenvalue, so the gradient of this point will shadow 
#  other data points and their gradients won't get updated, thus slowing the convergence

x_copy = x.copy()

for i in range(x.shape[1]):
  x_copy[:,i] = (x[:,i] - np.mean(x[:,i])) / (np.std(x[:,i]))



In [0]:
### 80-20 split 


X_train,X_val,y_train,y_val = train_test_split(x_copy, y, test_size=0.20, random_state=25)

In [0]:
vecy_train = np.zeros((6724,3))
for i in range(6724):
  vecy_train[i,y_train[i]] = 1 
  
vecy_val = np.zeros((1682,3))
for i in range(1682):
  vecy_val[i,y_val[i]] = 1 

In [0]:
def sigmoid(array):
  return (1 / (1 + np.exp(-array)))


def sigmoid_derivative(array):
  return sigmoid(array) * (1 - sigmoid(array))


def softmax(array):
  numerator   = np.exp(array)
  denominator = np.sum(numerator,axis = 1,keepdims=True)
  return numerator / denominator



In [0]:
## number of Layers


layers = [23,30,25,15,3]


#23 is input layer
#3 is output layer

In [0]:
np.random.seed(3)

In [0]:


linear_Z    = {}
nonlinear_A = {}
weights     = {}
gradients   = {}
noise       = {}

def initialize_layers(layers):
  for i in range(0,len(layers)-1):
    if(i == 0):
      weights['W'+str(i+1)] = np.random.randn(layers[0],layers[1]) 
      #weights['W'+str(i+1)] = np.random.uniform(low=-0.5, high=0.5, size=(layers[0],layers[1]))
      noise['b'+str(i+1)] = np.random.randn(layers[1]) * 0.1
    else:
      weights['W'+str(i+1)] = np.random.randn(layers[i],layers[i+1]) 
      #weights['W'+str(i+1)] = np.random.uniform(low=-0.5, high=0.5, size=(layers[i],layers[i+1]))
      noise['b'+str(i+1)] = np.random.randn(layers[i+1]) * 0.1


def linear_nonlinear_activation(layers,X_train,weights,linear_Z,nonlinear_A):
  for i in range(0,len(layers)-2):
    if(i == 0):
      noise['b'+str(i+1)] = np.random.randn(layers[1]) * 0.1
      linear_Z['Z'+str(i+1)] = np.dot(X_train, weights["W1"]) + noise['b1']
      nonlinear_A['A'+str(i+1)] = sigmoid(linear_Z['Z' + str(i+1)])

    else:
      noise['b'+str(i+1)] = np.random.randn(layers[i+1]) * 0.1
      linear_Z['Z'+str(i+1)] = np.dot(nonlinear_A['A'+str(i)],weights['W'+str(i+1)]) + noise['b'+str(i+1)]
      nonlinear_A['A'+str(i+1)] = sigmoid(linear_Z['Z' + str(i+1)])


def backward(X_train,dZ3,weights,gradients,linear_Z,nonlinear_A,layers):
  
  
  for i in range(len(layers)-1,1,-1):
    if(i == len(layers)-1):
      dZ_X = np.dot(dZ3,weights["W"+str(i)].T) * sigmoid_derivative(linear_Z["Z"+ str(i-1)])
      gradients["dW"+str(i-1)] = np.dot(nonlinear_A["A"+str(i-2)].T,dZ_X)
      
    elif(i == 2):
      dZ_X = np.dot(dZ_X,weights["W"+str(i)].T) * sigmoid_derivative(linear_Z["Z"+ str(i-1)])
      gradients["dW"+str(i-1)] = np.dot(X_train.T,dZ_X)
      
    else:
      dZ_X = np.dot(dZ_X,weights["W"+str(i)].T) * sigmoid_derivative(linear_Z["Z"+ str(i-1)])
      gradients["dW"+str(i-1)] = np.dot(nonlinear_A["A"+str(i-2)].T,dZ_X)
      

In [0]:
def moment_initializer(weights) :

    
    first_moment = {}
    second_moment = {}

    for i in range(1,len(weights)+1):
        first_moment["dW" + str(i)] = np.zeros((weights["W" + str(i)].shape[0],weights["W" + str(i)].shape[1]))

        second_moment["dW" + str(i)] = np.zeros((weights["W" + str(i)].shape[0],weights["W" + str(i)].shape[1]))

        
    
    return first_moment, second_moment

In [0]:
#initilization
initialize_layers(layers)

first_moment, second_moment = moment_initializer(weights)

radam_params ={'beta1':0.9,
              'beta2':0.99,
              'smoothing_term':1e-8,
              'learning_rate':0.0009,
              'time_step':0,
               'p_time':0,
              'first_moment':first_moment,
              'second_moment':second_moment}
p_max = (2 / (1 - radam_params['beta2']) ) - 1
radam_params['p_max'] = p_max

In [0]:
def radam(weights, gradients, first_moment, second_moment,p_max, time_step, learning_rate, beta1, beta2, smoothing_term=1e-8):

    
    first_moment_bias_correction = {}  
    second_moment_bias_correction = {}           
    
    
    p_time = p_max - ((( 2 * time_step * np.power(beta2,time_step))) / (1 - np.power(beta2,time_step)))
    #print(p_time)
    for i in range(1,len(weights)+1):

        first_moment["dW" + str(i)] = beta1 * first_moment["dW" + str(i)] + (1 - beta1) * gradients['dW' + str(i)]
        first_moment_bias_correction["dW" + str(i)] = first_moment["dW" + str(i)] / (1 - np.power(beta1, time_step))

     
        second_moment["dW" + str(i)] = beta2 * second_moment["dW" + str(i)] + (1 - beta2) * np.power(gradients['dW' + str(i)], 2)
        
        numerator = first_moment_bias_correction["dW" + str(i)]
        if(p_time > 4):
          second_moment_bias_correction["dW" + str(i)] = second_moment["dW" + str(i)] / (1 - np.power(beta2, time_step))
          denominator  = np.sqrt(second_moment_bias_correction["dW" + str(i)] + smoothing_term)
          r = ((p_time - 4)*(p_time-2)*p_max) / ((p_max-4)*(p_max - 2)*p_time)
          weights["W" + str(i)] = weights["W" + str(i)] - ((learning_rate * r* numerator) / (denominator))  

        else:
          weights["W" + str(i)] = weights["W" + str(i)] - ((learning_rate * numerator))
      
          
    
        
    
      
   

    return weights, first_moment, second_moment

In [0]:
def train_acc():
  c = 0
  linear_nonlinear_activation(layers,X_train,weights,linear_Z,nonlinear_A)
  
  linear_Z['Z'+str(len(weights))] = np.dot(nonlinear_A["A"+str(len(weights)-1)], weights['W'+str(len(weights))]) + noise['b'+str(len(noise))]
  nonlinear_A["A"+str(len(weights))] = softmax(linear_Z['Z'+str(len(weights))])
    
  
  for i in range(X_train.shape[0]):
    if(np.argmax(nonlinear_A["A"+str(len(weights))][i]) == y_train[i]):
      c = c+1
  return c/X_train.shape[0], np.sum(-vecy_train * np.log(nonlinear_A['A'+str(len(weights))]))

In [0]:
def val_acc():
  c = 0
  linear_nonlinear_activation(layers,X_val,weights,linear_Z,nonlinear_A)
  
  linear_Z['Z'+str(len(weights))] = np.dot(nonlinear_A["A"+str(len(weights)-1)], weights['W'+str(len(weights))])
  nonlinear_A["A"+str(len(weights))] = softmax(linear_Z['Z'+str(len(weights))])
    
  
  for i in range(X_val.shape[0]):
    if(np.argmax(nonlinear_A["A"+str(len(weights))][i]) == y_val[i]):
      c = c+1
  return c/X_val.shape[0], np.sum(-vecy_val * np.log(nonlinear_A['A'+str(len(weights))]))

In [0]:
cost_function_train = []
cost_function_val = []

def train(layers,X_train,vecy_train,weights,gradients,linear_Z,nonlinear_A,radam_params,cost_function_train,cost_function_val,total_epochs):
  
  for k in range(total_epochs):
  
    #Forward Prop
    linear_nonlinear_activation(layers,X_train,weights,linear_Z,nonlinear_A)
    
    
    linear_Z['Z'+str(len(weights))] = np.dot(nonlinear_A["A"+str(len(weights)-1)], weights['W'+str(len(weights))]) + noise['b'+str(len(noise))]
    nonlinear_A["A"+str(len(weights))] = softmax(linear_Z['Z'+str(len(weights))])
      
      
    #Backprop
    dZ3 = nonlinear_A["A"+str(len(weights))] - vecy_train
    gradients["dW"+str(len(weights))] = np.dot(nonlinear_A["A"+str(len(weights)-1)].T, dZ3)
    backward(X_train,dZ3,weights,gradients,linear_Z,nonlinear_A,layers)
    
    radam_params["time_step"] = radam_params["time_step"] + 1
    
    
    weights, radam_params["first_moment"], radam_params["second_moment"] = radam(weights, gradients, 
                                                                         radam_params["first_moment"], radam_params["second_moment"],
                                                                         radam_params['p_max'],
                                                                         radam_params["time_step"], radam_params["learning_rate"],
                                                                         radam_params["beta1"],radam_params["beta2"],
                                                                         
                                                                         radam_params["smoothing_term"])

    if k % 100 == 0:
        print("Epoch %d/%d "%(k,total_epochs),end = ' ')
        train_accuracy,train_loss = train_acc()
        val_accuracy,val_loss = val_acc()
        print('Loss: %f - train accuracy: %f - val_loss: %f - test/val accuracy: %f ' % (train_loss , train_accuracy ,val_loss,val_accuracy))
        cost_function_train.append(train_loss)
        cost_function_val.append(val_loss)

  return cost_function_train,cost_function_val 
    
    
    

In [28]:
loss_train,loss_val = train(layers,X_train,vecy_train,weights,gradients,linear_Z,nonlinear_A,radam_params,cost_function_train,cost_function_val,
                     total_epochs = 2301)

Epoch 0/2301  Loss: 48403.950374 - train accuracy: 0.658090 - val_loss: 11980.654949 - test/val accuracy: 0.662901 
Epoch 100/2301  Loss: 9221.600292 - train accuracy: 0.644557 - val_loss: 2329.431496 - test/val accuracy: 0.645065 
Epoch 200/2301  Loss: 8643.924810 - train accuracy: 0.658090 - val_loss: 2166.490730 - test/val accuracy: 0.662901 
Epoch 300/2301  Loss: 7965.891703 - train accuracy: 0.658090 - val_loss: 2008.602077 - test/val accuracy: 0.662901 
Epoch 400/2301  Loss: 7296.907440 - train accuracy: 0.771267 - val_loss: 1856.129227 - test/val accuracy: 0.693222 
Epoch 500/2301  Loss: 6801.887310 - train accuracy: 0.846669 - val_loss: 1736.981847 - test/val accuracy: 0.843044 
Epoch 600/2301  Loss: 6389.196226 - train accuracy: 0.848453 - val_loss: 1632.134657 - test/val accuracy: 0.846611 
Epoch 700/2301  Loss: 5737.175343 - train accuracy: 0.845925 - val_loss: 1477.335887 - test/val accuracy: 0.843639 
Epoch 800/2301  Loss: 4830.741223 - train accuracy: 0.849941 - val_loss: